# Setup

In [1]:
using ForwardDiff

# Using ForwardDiff to automatically differentiate functions

## Jacobians

Let's define a linear function:

In [2]:
const A = [1. 2.;
           2. 3.]
flinear(x) = A * x

flinear (generic function with 1 method)

ForwardDiff allows you to compute the Jacobian of this function *at a specific value of the arguments*:

In [3]:
ForwardDiff.jacobian(flinear, zeros(2))

2×2 Array{Float64,2}:
 1.0  2.0
 2.0  3.0

And here's a nonlinear function:

In [4]:
fnonlinear(x) = A * sqrt.(x) + 4 * x.^3

fnonlinear (generic function with 1 method)

In [5]:
ForwardDiff.jacobian(fnonlinear, [1., 2.])

2×2 Array{Float64,2}:
 12.5   0.707107
  1.0  49.0607  

## Hessians

Here's a quadratic form:

In [6]:
fquadratic(x) = 1/2 * dot(x, A * x)

fquadratic (generic function with 1 method)

ForwardDiff allows you to compute the Hessian automatically:

In [7]:
ForwardDiff.hessian(fquadratic, zeros(2))

2×2 Array{Float64,2}:
 1.0  2.0
 2.0  3.0

# How does it work?

ForwardDiff implements forward-mode [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).

Automatic differentiation is **not**:
* numeric differentiation
* symbolic differentiation

Automatic differentiation tools compute derivatives **without numerical approximation**, but **only at a specific point**.

They do so by:
* implementing a new 'scalar' type (similar to the standard `Float64`), and 
* overloading basic methods to implement the elementary rules of differentiation.

`Dual` is ForwardDiff's automatic differentiation scalar type:

In [8]:
dump(ForwardDiff.Dual{Float64, Float64, 3})

ForwardDiff.Dual{Float64,Float64,3} <: Real
  value::Float64
  partials::ForwardDiff.Partials{3,Float64}


Let's create some `ForwardDiff.Dual`s:

In [9]:
using ForwardDiff: Dual, Partials
x = Dual(1., Partials((1., 0.)))
y = Dual(2., Partials((0., 2.)))

Dual{Void}(2.0,0.0,2.0)

and add them together:

In [10]:
x + y

Dual{Void}(3.0,1.0,2.0)

In [11]:
@which x + y

+(x::ForwardDiff.Dual{T,V,N} where N where V<:Real, y::ForwardDiff.Dual{T,V,N} where N where V<:Real) where T in ForwardDiff at /Users/rdeits/Downloads/agile-test/packages/v0.6/ForwardDiff/src/dual.jl:93

Here's another example:

In [12]:
exp(x)

Dual{Void}(2.718281828459045,2.718281828459045,0.0)

In [13]:
@which exp(x)

exp(d::ForwardDiff.Dual{T,V,N} where N where V<:Real) where T in ForwardDiff at /Users/rdeits/Downloads/agile-test/packages/v0.6/ForwardDiff/src/dual.jl:168

Implements:

$$
\frac{\partial e^{x(z)}}{\partial z} = \frac{\partial e^{x}}{\partial x} \frac{\partial x}{\partial z} = e^x \frac{\partial x}{\partial z}
$$

*Reverse*-mode automatic differentiation is also available, e.g. in the form of [ReverseDiff.jl](https://github.com/JuliaDiff/ReverseDiff.jl).